## Import libraries

In [ ]:
import os
import cv2
from pydub import AudioSegment

### Function: `block_difference`
- Compares two frames by dividing them into blocks
- Calculates the mean absolute difference between corresponding blocks 
- Returns the average block difference across the whole frame

In [3]:
def block_difference(frame1, frame2, block_size=16):
    height, width = frame1.shape
    total_diff = 0
    num_blocks = 0

    for y in range(0, height, block_size):
        for x in range(0, width, block_size):
            block1 = frame1[y:y+block_size, x:x+block_size]
            block2 = frame2[y:y+block_size, x:x+block_size]
            if block1.shape == block2.shape:
                diff = np.abs(block1.astype(int) - block2.astype(int)).mean()
                total_diff += diff
                num_blocks += 1

    return total_diff / num_blocks if num_blocks > 0 else 0

### Function: `extract_keyframes_from_shot`
- Extracts keyframes from a list of frames using block difference
- Uses the first frame as reference, then adds new frames when difference exceeds a threshold
- Resizes keyframes to 224x224
- Ensures exactly 15 keyframes by padding with the last frame if needed

In [5]:
def extract_keyframes_from_shot(frames, fps, block_threshold=20):
    duration = len(frames) / fps
    target_num_keyframes = 15
    keyframes = []

    if len(frames) == 0:
        return []

    ref_gray = cv2.cvtColor(frames[0], cv2.COLOR_BGR2GRAY)
    resized_frame = cv2.resize(frames[0], (224, 224))
    keyframes.append(resized_frame)

    for i, frame in enumerate(frames[1:], start=1):
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        diff = block_difference(ref_gray, gray)
        if diff > block_threshold:
            resized = cv2.resize(frame, (224, 224))
            keyframes.append(resized)

        if len(keyframes) >= target_num_keyframes:
            break

    while len(keyframes) < target_num_keyframes:
        keyframes.append(keyframes[-1].copy())

    return keyframes


### Function: `extract_audio_shot`
- Extracts a segment of audio from a given audio file
- Takes `start_time` and `end_time` in seconds
- Saves the extracted audio segment as a `.wav` file

In [1]:
def extract_audio_shot(audio_path, start_time, end_time, save_path):
    audio = AudioSegment.from_file(audio_path)
    start_ms = int(start_time * 1000)
    end_ms = int(end_time * 1000)
    segment = audio[start_ms:end_ms]
    segment.export(save_path, format="wav")